**Aufgabe**
 Erzeuge neue Aggregationen damit folgende Facetten entstehen können: 
 
 - (1) Die Gruppierung nach Eigentümern (Vor- und Nachname) (2 Punkte)
 - (2) Die Gruppierung nach Speicherorten (Root-Folder). (2 Punkte)
 - (3) Die Gruppierung nach Upload-Datum. Flexibel nach Tag, Monat oder Jahr. (4 Punkte)
 - (4) Eine Gruppierung nach Dateiendungen. (2 Punkte)
 
 **Abgabe**
 
 Als Abgabe zählt das Vorzeigen oder Zuschicken der erzeugten Aggregations- Queries.

#### Unser Mapping:
 - ownername (keyword)
 - viewable (long)
 - public (boolean)
 - filepath (text)
 - filename (text)
 - filetype (text)
 - upload_day (short)
 - upload_month (short)
 - upload_year (short)


In [1]:
# Delete Created Data
curl -XPOST 'localhost:9200/htwplus/medium/_delete_by_query?conflicts=proceed&pretty' -H 'Content-Type: application/json' -d'
{
  "query": {
    "match_all": {}
  }
}
'

{
  "took" : 16,
  "timed_out" : false,
  "total" : 0,
  "deleted" : 0,
  "batches" : 0,
  "version_conflicts" : 0,
  "noops" : 0,
  "retries" : {
    "bulk" : 0,
    "search" : 0
  },
  "throttled_millis" : 0,
  "requests_per_second" : -1.0,
  "throttled_until_millis" : 0,
  "failures" : [ ]
}


In [2]:
# Generate a few data samples to play with
curl -XPOST 'localhost:9200/htwplus/medium?pretty' -H 'Content-Type: application/json' -d'
{
    "ownername": "Robin Wenzel",
    "viewable": 1,
    "public": false,
    "filepath": "media/docs/robin/",
    "filename": "bachelor.pdf",
    "filetype": "pdf",
    "upload_day": 15,
    "upload_month": 7,
    "upload_year": 2017,
    "upload_date": "2017-07-15"
}'

curl -XPOST 'localhost:9200/htwplus/medium?pretty' -H 'Content-Type: application/json' -d'
{
    "ownername": "Robin Wenzel",
    "viewable": 1,
    "public": false,
    "filepath": "media/docs/robin/",
    "filename": "avatar.png",
    "filetype": "png",
    "upload_day": 15,
    "upload_month": 8,
    "upload_year": 2016,
    "upload_date": "2016-08-15"
}'

curl -XPOST 'localhost:9200/htwplus/medium?pretty' -H 'Content-Type: application/json' -d'
{
    "ownername": "Iven Zimmer",
    "viewable": 1,
    "public": false,
    "filepath": "media/docs/iven/",
    "filename": "klausur.pdf",
    "filetype": "pdf",
    "upload_day": 14,
    "upload_month": 8,
    "upload_year": 2017,
    "upload_date": "2017-07-14"
}'

curl -XPOST 'localhost:9200/htwplus/medium?pretty' -H 'Content-Type: application/json' -d'
{
    "ownername": "Sebastian Schmid",
    "viewable": 1,
    "public": true,
    "filepath": "media/docs/iven/",
    "filename": "presentation.pdf",
    "filetype": "pdf",
    "upload_day": 14,
    "upload_month": 9,
    "upload_year": 2017,
    "upload_date": "2017-09-14"
}'

{
  "_index" : "htwplus",
  "_type" : "medium",
  "_id" : "AV1h7YlyVByKtHw0fYXh",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "created" : true
}
{
  "_index" : "htwplus",
  "_type" : "medium",
  "_id" : "AV1h7YxVVByKtHw0fYXi",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "created" : true
}
{
  "_index" : "htwplus",
  "_type" : "medium",
  "_id" : "AV1h7Y8wVByKtHw0fYXj",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "created" : true
}
{
  "_index" : "htwplus",
  "_type" : "medium",
  "_id" : "AV1h7ZIJVByKtHw0fYXk",
  "_version" : 1,
  "result" : "created",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "created" : true
}


In [3]:
# Task1: Aggregate by Owner -> Create Buckets
# Only Output result with > "size": 0 <
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs" : {
        "owners" : {
            "terms" : { "field": "ownername" }
        }
    }
}'

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "owners" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : "Robin Wenzel",
          "doc_count" : 2
        },
        {
          "key" : "Iven Zimmer",
          "doc_count" : 1
        },
        {
          "key" : "Sebastian Schmid",
          "doc_count" : 1
        }
      ]
    }
  }
}


In [4]:
# Task2: Aggregate by FilePath -> Create Buckets
# Only Output result with > "size": 0 <
# Need to enable fielddata: true in mapping ..!
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "paths": {
            "terms" : { "field": "filepath" }
        }
    }
}'

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "paths" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : "media",
          "doc_count" : 4
        },
        {
          "key" : "media/docs",
          "doc_count" : 4
        },
        {
          "key" : "media/docs/iven",
          "doc_count" : 2
        },
        {
          "key" : "media/docs/iven/",
          "doc_count" : 2
        },
        {
          "key" : "media/docs/robin",
          "doc_count" : 2
        },
        {
          "key" : "media/docs/robin/",
          "doc_count" : 2
        }
      ]
    }
  }
}


In [5]:
# Aggregate by Upload: Day
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "by_days": {
            "terms" : { "field": "upload_day" }
        }
    }
}'

{
  "took" : 3,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "by_days" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : 14,
          "doc_count" : 2
        },
        {
          "key" : 15,
          "doc_count" : 2
        }
      ]
    }
  }
}


In [6]:
# Aggregate by Upload: Month
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "by_month": {
            "terms" : { "field": "upload_month" }
        }
    }
}'

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "by_month" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : 8,
          "doc_count" : 2
        },
        {
          "key" : 7,
          "doc_count" : 1
        },
        {
          "key" : 9,
          "doc_count" : 1
        }
      ]
    }
  }
}


In [7]:
# Aggregate by Upload: Month
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "by_year": {
            "terms" : { "field": "upload_year" }
        }
    }
}'

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "by_year" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : 2017,
          "doc_count" : 3
        },
        {
          "key" : 2016,
          "doc_count" : 1
        }
      ]
    }
  }
}


In [8]:
# Aggregate by Upload: Day / Year
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "years": {
            "terms" : { "field": "upload_year" },
            "aggs": {
                "days": { "terms" : { "field": "upload_day" } }
            }
        }
    }
}'

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "years" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : 2017,
          "doc_count" : 3,
          "days" : {
            "doc_count_error_upper_bound" : 0,
            "sum_other_doc_count" : 0,
            "buckets" : [
              {
                "key" : 14,
                "doc_count" : 2
              },
              {
                "key" : 15,
                "doc_count" : 1
              }
            ]
          }
        },
        {
          "key" : 2016,
          "doc_count" : 1,
          "days" : {
            "doc_count_error_upper_bound" : 0,
            "sum_other_doc_count" : 0,
            "buckets" : [
              {
                "key" : 15,
                "doc_cou

In [9]:
# Aggregate by Upload: Year/Month/Day
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "years": {
            "terms" : { "field": "upload_year" },
            "aggs": {
                "months": {
                    "terms" : { "field": "upload_month" },
                    "aggs": {
                        "days": {
                        "terms" : { "field": "upload_month" }
                        }
                    }
                }
            }
        }
    }
}'

{
  "took" : 2,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "years" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : 2017,
          "doc_count" : 3,
          "months" : {
            "doc_count_error_upper_bound" : 0,
            "sum_other_doc_count" : 0,
            "buckets" : [
              {
                "key" : 7,
                "doc_count" : 1,
                "days" : {
                  "doc_count_error_upper_bound" : 0,
                  "sum_other_doc_count" : 0,
                  "buckets" : [
                    {
                      "key" : 7,
                      "doc_count" : 1
                    }
                  ]
                }
              },
              {
                "key" : 8,
                "doc_count" : 1,
   

In [10]:
# Aggregate by Filetype
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "by_ftype": {
            "terms" : { "field": "filetype" }
        }
    }
}'

{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "by_ftype" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : "pdf",
          "doc_count" : 3
        },
        {
          "key" : "png",
          "doc_count" : 1
        }
      ]
    }
  }
}


In [4]:
# Aggregate by Upload: Year/Month/Day
curl -XPOST 'localhost:9200/htwplus/medium/_search?pretty' -H 'Content-Type: application/json' -d'
{
    "size": 0,
    "aggs": {
        "years": {
            "filter":{
                "range": {"upload_date": { "gte": "2017" }}
            },
            "aggs": {
                "months": {
                    "terms" : { "field": "upload_month" },
                    "aggs": {
                        "days": {
                        "terms" : { "field": "upload_day" }
                        }
                    }
                }
            }
        }
    }
}'

{
  "took" : 9,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "failed" : 0
  },
  "hits" : {
    "total" : 4,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "aggregations" : {
    "years" : {
      "doc_count" : 3,
      "months" : {
        "doc_count_error_upper_bound" : 0,
        "sum_other_doc_count" : 0,
        "buckets" : [
          {
            "key" : 7,
            "doc_count" : 1,
            "days" : {
              "doc_count_error_upper_bound" : 0,
              "sum_other_doc_count" : 0,
              "buckets" : [
                {
                  "key" : 7,
                  "doc_count" : 1
                }
              ]
            }
          },
          {
            "key" : 8,
            "doc_count" : 1,
            "days" : {
              "doc_count_error_upper_bound" : 0,
              "sum_other_doc_count" : 0,
              "buckets" : [
                {
                  "key" : 8,
                  "doc_count" :